In [1]:
from __future__ import division, absolute_import

import sys
import random
import pickle

from sklearn import metrics
import pandas as pd
import numpy as np
import h5py
from plotnine import *

#root
absPath = '/home/angela3/imbalance_pcm_benchmark/'
sys.path.insert(0, absPath)

from src.Target import Target
from src.imbalance_functions import *

np.random.seed(8)
random.seed(8)

Using TensorFlow backend.


In [2]:
protein_type = "GPCRs" #"kinases"
nfolds = 10

group = '/activity'
table = "prot_comp"

#Loading maximum lengths of proteins and compounds
with open("".join((absPath, 'data/prot_max_len.pickle')), "rb") as input_file:
    max_len_prot = pickle.load(input_file)
#Defining protein dictionary    
instarget = Target("AAA")
prot_dict = instarget.predefining_dict()

with open("".join((absPath, "data/", protein_type, "/", protein_type, "_prots.pickle")), 'rb') as handle:
    unique_prots = pickle.load(handle)

In [3]:
protein_info = "".join((absPath, "data/", protein_type, "/prots_df.csv"))
prot_df = pd.read_csv(protein_info)
prot_df.drop(["Unnamed: 0"],axis=1, inplace=True)
prot_df.head()
prot_df["len_seq"] = prot_df.apply(len_seq, axis=1)

In [4]:
ratios_per_fold = []
for fold in range(nfolds):
    f_train = h5py.File("".join((absPath, "data/", protein_type, "/resampling_after_clustering/",
                                 str(fold), "/compounds_activity_training.h5")), "r")
    
    f_test = h5py.File("".join((absPath, "data/", protein_type, "/resampling_after_clustering/",
                                 str(fold), "/compounds_activity_test.h5")), "r")
    
    sample_indices_train = range(len(f_train[group][table]))
    sample_indices_test = range(len(f_test[group][table]))
    
    training_ratios = creating_ratios_list("training", fold, f_train, group, table, sample_indices_train,
                                           unique_prots)
    test_ratios = creating_ratios_list("test", fold, f_test, group, table, sample_indices_test,
                                       unique_prots)
    
    pred_test_path = "".join((absPath, "data/", protein_type, "/resampling_after_clustering/predictions/", 
                              str(fold), "/test.csv"))
    pred_test = pd.read_csv(pred_test_path)
    pred_test_ratios, pred_test_ = predictions_ratios_list(fold, pred_test, unique_prots)
    
    list_metrics = []
    for prot in unique_prots:
        dict_prot = computing_metrics_per_prot(prot, pred_test_)
        list_metrics.append(dict_prot)
        
    filt_list_metrics = []
    for val in list_metrics:
        if val != None:
            filt_list_metrics.append(val)
        
    ratios_df = converting_ratios_to_df(training_ratios, test_ratios, pred_test_ratios, filt_list_metrics,
                                        "resampling_after_clustering", prot_df)
    ratios_df["fold"] = str(fold)
    ratios_per_fold.append(ratios_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239816 entries, 0 to 239815
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   DeepAffinity Protein ID  239816 non-null  object
 1   label                    239816 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28980 entries, 0 to 28979
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DeepAffinity Protein ID  28980 non-null  object
 1   label                    28980 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 452.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28980 entries, 0 to 28979
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label                  

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 non-null    object 
 1   ratio_training           285 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 non-null    object 
 1   ratio_test               135 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 non-null    object 
 1   ratio_training           292 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 non-null    object 
 1   ratio_test               120 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 

...

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 non-null    object 
 1   ratio_training           287 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 non-null    object 
 1   ratio_test               120 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241528 entries, 0 to 241527
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   DeepAffinity Protein ID  241528 non-null  object
 1   label                    241528 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29658 entries, 0 to 29657
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DeepAffinity Protein ID  29658 non-null  object
 1   label                    29658 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 463.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29658 entries, 0 to 29657
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label                  

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 non-null    object 
 1   ratio_training           290 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 non-null    object 
 1   ratio_test               122 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  498 

In [5]:
ratios_complete_df = pd.concat(ratios_per_fold)
ratios_complete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1203 entries, 0 to 121
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  1203 non-null   object 
 1   ratio_training           1182 non-null   float64
 2   ratio_test               1203 non-null   float64
 3   ratio_test_predicted     1203 non-null   float64
 4   acc                      1203 non-null   float64
 5   auroc                    1203 non-null   float64
 6   f1                       1203 non-null   float64
 7   balanced_acc             1203 non-null   float64
 8   mcc                      1203 non-null   float64
 9   strategy                 1203 non-null   object 
 10  fold                     1203 non-null   object 
dtypes: float64(8), object(3)
memory usage: 112.8+ KB


In [6]:
ratios_seq = pd.merge(ratios_complete_df, prot_df, "left", on="DeepAffinity Protein ID")
print(ratios_seq.info())
print(ratios_seq.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1203 entries, 0 to 1202
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  1203 non-null   object 
 1   ratio_training           1182 non-null   float64
 2   ratio_test               1203 non-null   float64
 3   ratio_test_predicted     1203 non-null   float64
 4   acc                      1203 non-null   float64
 5   auroc                    1203 non-null   float64
 6   f1                       1203 non-null   float64
 7   balanced_acc             1203 non-null   float64
 8   mcc                      1203 non-null   float64
 9   strategy                 1203 non-null   object 
 10  fold                     1203 non-null   object 
 11  Sequence                 1203 non-null   object 
 12  family                   1203 non-null   object 
 13  Uniprot ID               1203 non-null   object 
 14  n_interactions          

In [7]:
ratios_seq.to_csv("".join((absPath, "data/", protein_type, "/resampling_after_clustering/results/ratios_df.csv")))